For at køre følgende notesbøger, hvis du ikke allerede har gjort det, skal du angive openai-nøglen i .env-filen som `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Dernæst skal vi indlæse Embedding Index i en Pandas Dataframe. Embedding Index er gemt i en JSON-fil kaldet `embedding_index_3m.json`. Embedding Index indeholder Embeddings for hver af YouTube-transkriptionerne frem til slutningen af oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Næste skal vi oprette en funktion kaldet `get_videos`, som vil søge i Embedding Index efter forespørgslen. Funktionen returnerer de 5 videoer, der minder mest om forespørgslen. Funktionen fungerer således:

1. Først oprettes en kopi af Embedding Index.
2. Dernæst beregnes Embedding for forespørgslen ved hjælp af OpenAI Embedding API.
3. Så oprettes en ny kolonne i Embedding Index kaldet `similarity`. Kolonnen `similarity` indeholder cosinus-ligheden mellem forespørgslens Embedding og Embedding for hvert videosegment.
4. Herefter filtreres Embedding Index efter kolonnen `similarity`. Embedding Index filtreres, så kun videoer med en cosinus-lighed på mindst 0,75 medtages.
5. Til sidst sorteres Embedding Index efter kolonnen `similarity`, og de 5 bedste videoer returneres.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Denne funktion er meget enkel, den udskriver blot resultaterne af søgeforespørgslen.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Først bliver Embedding Index indlæst i en Pandas Dataframe.
2. Derefter bliver brugeren bedt om at indtaste en forespørgsel.
3. Så bliver funktionen `get_videos` kaldt for at søge i Embedding Index efter forespørgslen.
4. Til sidst bliver funktionen `display_results` kaldt for at vise resultaterne til brugeren.
5. Brugeren bliver derefter bedt om at indtaste en ny forespørgsel. Denne proces fortsætter, indtil brugeren skriver `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.da.png)

Du vil blive bedt om at indtaste en forespørgsel. Indtast en forespørgsel og tryk enter. Applikationen returnerer en liste over videoer, der er relevante for forespørgslen. Applikationen giver også et link til det sted i videoen, hvor svaret på spørgsmålet findes.

Her er nogle forespørgsler, du kan prøve:

- Hvad er Azure Machine Learning?
- Hvordan fungerer convolutional neural networks?
- Hvad er et neuralt netværk?
- Kan jeg bruge Jupyter Notebooks med Azure Machine Learning?
- Hvad er ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel, menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå ved brug af denne oversættelse.
